In [1]:
from PARAM import *

from math import pi

import csv
import numpy as np
import matplotlib.pyplot as plt


from model_Trains import Trains
from model_Aproximators import Aproximators
from model_DataGenerator import DataGenerator

from tqdm import tqdm

from Base import *
from testTrains import testTrains

from model_Trace import Trace

import os
import csv

In [2]:
steps = 1000

trains = [0,1,2,3]
dop1 = [0 for i in trains]
dop2 = [0 for i in trains]
maxEs = [0 for i in trains]
srEs = [0 for i in trains]
maxSKOs = [0 for i in trains]
SKOs = [0 for i in trains]
SKOs_S = [0 for i in trains]
times = [0 for i in trains]

In [3]:
for i in tqdm(range(steps)):
    orig, E2, E3, time = run(
        testTrains(1/10),
        DataGenerator["Generators"][1](
            epsU = epsU,
            epsZR = epsZR,
            epsZO = epsZO,
            seed = i
        ),
        steps = length
    )

    SKO_0 = (sum([e**2 for e in E2[0]])/len(E2[0]))**0.5
    for i, q in enumerate(zip(E2, time)):
        e2, t = q
        SKO = (sum([e**2 for e in e2])/len(e2))**0.5

        dop1[i] += 1 if SKO < SKO_0 else 0
        dop2[i] += 1 if sum(e2)/len(e2) < sum(E2[0])/len(E2[0]) else 0

        maxEs[i] = max(maxEs[i], max(e2))
        srEs[i] += sum(e2)/len(e2)
        maxSKOs[i] = max(maxSKOs[i], SKO)

        SKOs[i] += SKO
        SKOs_S[i] += SKO/orig[2]
        times[i] += sum(t)/len(t)

for i in range(len(maxEs)):
    srEs[i] /= steps
    SKOs[i] /= steps
    SKOs_S[i] /= steps
    times[i] /= steps

100%|██████████| 1000/1000 [21:08<00:00,  1.27s/it]


In [4]:
def printLine(line):
    print(line[0], end='\t')
    for n in line[1:]:
        print("{:.7}".format(n), end='\t')
    print()
    
with open("table_result.csv" , "w", encoding='cp1251',newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=';', dialect=csv.excel_tab)

    lines = [
        ["Алгоритм", "Оригинальный", "Фурье", "Равноускоренный", "Тряск"],
        ["Лучше оригинального по средним"] + [str(d).replace(".",",") for d in dop2],
        ["Лучше оригинального по СКО"] + [str(d).replace(".",",") for d in dop1],
        ["Максимальная ошибка, м"] + [str(d).replace(".",",") for d in maxEs],
        ["Средняя ошибка, м"] + [str(d).replace(".",",") for d in srEs],
        ["Максимальное СКО, м"] + [str(d).replace(".",",") for d in maxSKOs],
        ["Среднее СКО, м"] + [str(d).replace(".",",") for d in SKOs],
        ["Среднее отношение СКО/S"] + [str(d).replace(".",",") for d in SKOs_S],
        ["Среднее время итерации, мc"] + [str(d*1000).replace(".",",") for d in times]
    ]

    for line in lines:
        printLine(line)
        writer.writerow(line)

Алгоритм	Оригина	Фурье	Равноус	Тряск	
Лучше оригинального по средним	0	936	890	0	
Лучше оригинального по СКО	0	805	740	0	
Максимальная ошибка, м	0,12084	0,12174	0,12174	7068,49	
Средняя ошибка, м	0,01197	0,01195	0,01195	0,05380	
Максимальное СКО, м	0,03064	0,03070	0,03070	220,186	
Среднее СКО, м	0,01421	0,01420	0,01420	0,77900	
Среднее отношение СКО/S	0,00052	0,00052	0,00052	0,03130	
Среднее время итерации, мc	0,04718	0,14441	0,06259	0,09244	
